In [6]:
!pip install tensorflow numpy


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [8]:
# Download the IMDb dataset from keras.datasets
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# Pad sequences to a fixed length (e.g., 256)
max_len = 256
train_data = pad_sequences(train_data, maxlen=max_len, padding='post')
test_data = pad_sequences(test_data, maxlen=max_len, padding='post')

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42
)


In [9]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def create_transformer_model(max_len, vocab_size=10000, embed_dim=32, num_heads=2, ff_dim=32):
    inputs = layers.Input(shape=(max_len,))

    # Embedding layer
    embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)

    # Transformer Block
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)(embedding_layer)

    # Global Average Pooling 1D
    x = layers.GlobalAveragePooling1D()(transformer_block)

    # Dense layer and output layer
    x = layers.Dense(20, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model

# Create the transformer model
model = create_transformer_model(max_len=max_len)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
# Train the model
model.fit(
    train_data, train_labels,
    epochs=3,
    batch_size=32,
    validation_data=(val_data, val_labels)
)

Epoch 1/3
625/625 [==============================] - 99s 158ms/step - loss: 0.1576 - accuracy: 0.9423 - val_loss: 0.3150 - val_accuracy: 0.8796
Epoch 2/3
625/625 [==============================] - 100s 160ms/step - loss: 0.1210 - accuracy: 0.9591 - val_loss: 0.4270 - val_accuracy: 0.8750
Epoch 3/3
625/625 [==============================] - 107s 171ms/step - loss: 0.0945 - accuracy: 0.9694 - val_loss: 0.4857 - val_accuracy: 0.8690


In [12]:
# Evaluate on the test set
test_pred_probs = model.predict(test_data)
test_preds = (test_pred_probs > 0.5).astype(int)

# Calculate accuracy and other metrics
accuracy = accuracy_score(test_labels, test_preds)
classification_rep = classification_report(test_labels, test_preds)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_rep)


782/782 [==============================] - 51s 65ms/step
Accuracy: 0.8493
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.83      0.85     12500
           1       0.83      0.87      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

